In [ ]:
import pandas as pd
data = pd.read_excel('C:/Users/ASUS/OneDrive/Desktop/Drug_Data_With_Stock.xlsx')


In [ ]:
print(data.info())
print(data.describe())
print(data.head())


Visualizations and Analyses:
Time-Series Analysis:

Plot sales trends over time for different drugs.

In [ ]:
import matplotlib.pyplot as plt

for drug in data['Drug Name'].unique():
    drug_data = data[data['Drug Name'] == drug]
    plt.plot(drug_data['Date'], drug_data['Sales'], label=drug)

plt.xlabel('Date')
plt.ylabel('Sales')
plt.title('Sales Trends Over Time')
plt.legend()
plt.show()


Correlation Analysis:

Use a heatmap to explore correlations between numerical features.

In [ ]:
import seaborn as sns

numeric_data = data.select_dtypes(include=['float64', 'int64'])
correlation_matrix = numeric_data.corr()

plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()


Seasonality Check:

Check if there are seasonal patterns in sales.

In [ ]:
data['Month'] = pd.to_datetime(data['Date']).dt.month
monthly_sales = data.groupby('Month')['Sales'].sum()

plt.plot(monthly_sales.index, monthly_sales.values, marker='o')
plt.title('Monthly Sales Pattern')
plt.xlabel('Month')
plt.ylabel('Total Sales')
plt.xticks(range(1, 13))
plt.show()


Boxplots for Distribution:

Understand the distribution of sales, prices, etc., across different categories.

In [ ]:
sns.boxplot(x='Drug Name', y='Sales', data=data)
plt.xticks(rotation=45)
plt.title('Sales Distribution Across Drugs')
plt.show()


Feature Relationships:

Analyze relationships like sales vs. retail price or sales vs. buffer stock.

In [ ]:
sns.scatterplot(x='Retail Price', y='Sales', hue='Drug Name', data=data)
plt.title('Sales vs. Retail Price')
plt.show()
